In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Desactiva los mensajes de advertencia

In [ ]:
pip install "accelerate>=0.26.0"

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Cargar el dataset
dataset_path = '/content/news_pruebalisto.csv'
df = pd.read_csv(dataset_path, on_bad_lines='skip')

# Dividir en entrenamiento y validación
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label'].tolist()  # Añadir estratificación
)

# Tokenización
model_name = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Dataset personalizado
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item

train_dataset = FakeNewsDataset(train_encodings, train_labels)
val_dataset = FakeNewsDataset(val_encodings, val_labels)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Cargar el modelo
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    hidden_dropout_prob=0.4,  # Aumentar dropout
    attention_probs_dropout_prob=0.4
)

# Congelar capas iniciales
for param in model.bert.embeddings.parameters():
    param.requires_grad = False
for param in model.bert.encoder.layer[:6].parameters():
    param.requires_grad = False

# Métricas personalizadas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.softmax(torch.tensor(logits), dim=-1)[:, 1]
    threshold = 0.45
    binary_predictions = (predictions > threshold).int()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, binary_predictions, average='binary')
    acc = accuracy_score(labels, binary_predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",  # Cambiar a F1 score
    greater_is_better=True,
    learning_rate=5e-6,  # Reducir learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.1,  # Aumentar weight decay
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir='./logs'
)

# Early Stopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,  # Incrementar paciencia
    early_stopping_threshold=0.001
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# Entrenar
trainer.train()

# Evaluar
results = trainer.evaluate()
print("Resultados de evaluación:", results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.813151,0.528019,0.679528,0.514610,1.000000
2,0.622000,0.533655,0.714286,0.777368,0.637097,0.996845
3,0.622000,0.422898,0.801105,0.831325,0.722093,0.979495
4,0.502700,0.405487,0.808998,0.837147,0.730047,0.981073
5,0.449300,0.363568,0.827940,0.849655,0.754902,0.971609
6,0.449300,0.334807,0.847672,0.861847,0.788991,0.949527
7,0.426000,0.343572,0.838990,0.857542,0.769424,0.968454
8,0.398000,0.335906,0.839779,0.857344,0.773131,0.962145


Resultados de evaluación: {'eval_loss': 0.3348068296909332, 'eval_accuracy': 0.8476716653512234, 'eval_f1': 0.8618468146027202, 'eval_precision': 0.7889908256880734, 'eval_recall': 0.9495268138801262, 'eval_runtime': 8.7274, 'eval_samples_per_second': 145.175, 'eval_steps_per_second': 9.167, 'epoch': 8.0}
